# 4 Heston Model

Call Only

In [5]:
import numpy as np

def heston_option_price(S0, V0, r, k, theta, sigma, rho, T, K, n_steps=1000, n_paths=10000):
    """
    Calculates the price of a European call option using the Heston model.

    Parameters:
    S0 : float - Initial stock price
    V0 : float - Initial variance (volatility squared)
    r : float - Risk-free rate
    k : float - Speed of mean reversion of variance
    theta : float - Long-term variance
    sigma : float - Volatility of volatility
    rho : float - Correlation between the asset price and variance
    T : float - Time to maturity
    K : float - Strike price
    n_steps : int - Number of time steps (default: 1000)
    n_paths : int - Number of simulated paths (default: 10000)

    Returns:
    call_price : float - Estimated European call option price
    """
    dt = T / n_steps  # Time step size
    
    # Random numbers for the two Wiener processes
    dW1 = np.random.normal(0, np.sqrt(dt), (n_paths, n_steps))
    dW2 = np.random.normal(0, np.sqrt(dt), (n_paths, n_steps))

    # Correlate dW1 and dW2 using the correlation coefficient rho
    dW2 = rho * dW1 + np.sqrt(1 - rho**2) * dW2

    # Arrays to hold simulated asset prices and variances
    S = np.zeros((n_paths, n_steps + 1))
    V = np.zeros((n_paths, n_steps + 1))

    # Initial values
    S[:, 0] = S0
    V[:, 0] = V0

    # Simulating the paths using the Heston model
    for t in range(1, n_steps + 1):
        V[:, t] = np.maximum(V[:, t - 1] + k * (theta - V[:, t - 1]) * dt + sigma * np.sqrt(V[:, t - 1]) * dW2[:, t - 1], 0)
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * V[:, t - 1]) * dt + np.sqrt(V[:, t - 1]) * dW1[:, t - 1])

    # Payoff for a European Call Option (S_T - K)+
    call_payoff = np.maximum(S[:, -1] - K, 0)

    # Discounting the payoff to present value
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    print(f"Heston Call Option Price: {call_price:.2f}")

In [4]:
call_price = heston_option_price(S0 = 100, V0 = 0.04, r = 0.05, k = 2.0, theta = 0.04, sigma = 0.3, rho = -0.7, T = 1.0, K = 100)

Heston Call Option Price: 10.34
